In [334]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [335]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [336]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [337]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [364]:
bnf_1min = pd.read_csv("../data/ng_4h.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2012]

In [365]:
bnf_1min.head()

,time,open,high,low,close,datetime
0,2015-03-02T09:00:00+05:30,169.0000,169.0000,167.8000,168.3000,2015-03-02 09:00:00+05:30
1,2015-03-02T13:00:00+05:30,168.3000,168.8000,167.8000,168.4000,2015-03-02 13:00:00+05:30
2,2015-03-02T17:00:00+05:30,168.3000,171.7000,168.2000,170.9000,2015-03-02 17:00:00+05:30
3,2015-03-02T21:00:00+05:30,170.9000,171.7000,167.9000,169.1000,2015-03-02 21:00:00+05:30
4,2015-03-03T09:00:00+05:30,168.8000,168.8000,166.1000,166.6000,2015-03-03 09:00:00+05:30


In [366]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [367]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2017, 4, 11),
 datetime.date(2023, 8, 24),
 datetime.date(2017, 2, 2),
 datetime.date(2022, 6, 2),
 datetime.date(2015, 10, 23),
 datetime.date(2017, 8, 17),
 datetime.date(2016, 8, 2),
 datetime.date(2022, 8, 10),
 datetime.date(2021, 3, 17),
 datetime.date(2020, 3, 27),
 datetime.date(2024, 4, 11),
 datetime.date(2019, 11, 14),
 datetime.date(2023, 7, 28),
 datetime.date(2018, 7, 18),
 datetime.date(2024, 6, 10),
 datetime.date(2024, 6, 18),
 datetime.date(2022, 9, 22),
 datetime.date(2024, 8, 30),
 datetime.date(2016, 10, 28),
 datetime.date(2019, 2, 27),
 datetime.date(2021, 3, 12),
 datetime.date(2017, 10, 27),
 datetime.date(2021, 4, 20),
 datetime.date(2018, 4, 4),
 datetime.date(2018, 3, 29),
 datetime.date(2021, 4, 14),
 datetime.date(2021, 4, 21),
 datetime.date(2022, 5, 4),
 datetime.date(2020, 10, 1),
 datetime.date(2019, 12, 4),
 datetime.date(2020, 12, 14),
 datetime.date(2023, 10, 9),
 datetime.date(2018, 10, 1),
 datetime.date(2018, 10, 8),
 datetime.date

In [368]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [369]:
async def create_sell_signals(df):

    df["sell_signal"] = False

    df["sell_signal"] = (df["high"] < df["high"].shift(1)) & (
        df["high"].shift(2) < df["high"].shift(1)
    )

    return df

async def create_buy_signals(df):
    df["buy_signal"] = False

    df["buy_signal"] = (df["low"] > df["low"].shift(1)) & (
        df["low"].shift(2) > df["low"].shift(1)
    )

    return df

async def create_ma(df, period):
    df["close"] = pd.to_numeric(df["close"], errors="coerce")
    df["ma"] = df["close"].rolling(window=period).mean()
    return df

In [382]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.001
LEVERAGE_ = 5

In [569]:
def positional(df, sl_pct, target_pct):
    trades = []
    in_trade_long = False
    in_trade_short = False
    current_trade = None
    king_candle = None
    queen_candle = None
    slippage_pct = SLIPPAGE_
    portfolio = 10000000
    index_lev = 6
    entry_time = None
    exit_time = None
    is_trailing_active_long = False
    is_trailing_active_short = False
    points_long = None
    points_short = None
    initial_sl_long = 100000
    initial_sl_short = 0
    latest_king_found = False
    latest_queen_found = False

    # df['100ma'] = df['close'].rolling(100).mean()

    for i in range(3, len(df)):
        prev2_candle = df.iloc[i-3]
        prev_candle = df.iloc[i-2]
        last_candle = df.iloc[i-1]
        current_candle = df.iloc[i]

        if last_candle['high'] <= prev_candle['high'] and prev2_candle['high'] <= prev_candle['high']:
            king_candle = prev_candle
            print(f'King Candle : \n{df.iloc[i-2]}')

        if last_candle['low'] >= prev_candle['low'] and prev2_candle['low'] >= prev_candle['low']:
            queen_candle = prev_candle
            print(f'Queen Candle : \n{df.iloc[i-2]}')

        # Check for King candle formation when not in a trade
        if not in_trade_long and last_candle['high'] <= prev_candle['high'] and prev2_candle['high'] <= prev_candle['high']:
            king_candle = prev_candle
            signal_time_long = king_candle['datetime']
            # initial_sl_long = min(current_candle['low'], prev_candle['low'], prev2_candle['low'])
            initial_sl_long = queen_candle['low'] if (queen_candle is not None and queen_candle['low'] < king_candle['high']) else (min(last_candle['low'], prev_candle['low'], prev2_candle['low']))
            # print(f'King Candle Setup Complete : {df.iloc[i-1]["datetime"]}')

        # Check for Queen candle formation when not in a trade
        if not in_trade_short and last_candle['low'] >= prev_candle['low'] and prev2_candle['low'] >= prev_candle['low']:
            queen_candle = prev_candle
            signal_time_short = queen_candle['datetime']
            # initial_sl_short = max(current_candle['high'], prev_candle['high'], prev2_candle['high'])
            initial_sl_short = king_candle['high'] if (king_candle is not None and king_candle['high'] > queen_candle['low']) else (max(last_candle['high'], prev_candle['high'], prev2_candle['high']))
            # print(f'Queen Candle Setup Complete : {df.iloc[i-1]["datetime"]}')
            
        # Check for King breakout (Long position)
        if king_candle is not None and not in_trade_long and current_candle['high'] >= king_candle['high']:
            
            if (current_candle['open'] > king_candle['high'] and current_candle['low'] <= king_candle['high']) or (current_candle['open'] <= king_candle['high']):
                entry_price_long = king_candle['high']
                stop_loss_long = queen_candle['low'] if (queen_candle is not None and queen_candle['low'] < king_candle['high']) else (min(last_candle['low'], prev_candle['low'], prev2_candle['low']))
                # stop_loss_long = entry_price_long * (1 - sl_pct)
                # stop_loss_long = initial_sl_long
                # target_long = entry_price_long * (1 + target_pct)
                entry_time_long = df.iloc[i]['datetime']
                in_trade_long = True
            
            elif current_candle['low'] > king_candle['high']:
                king_candle = None
                in_trade_long = False

        if in_trade_long:

            if last_candle['high'] <= prev_candle['high'] and prev2_candle['high'] <= prev_candle['high']:
                # King Candle Formed, Trail SL
                latest_king_found = True

            if (last_candle['low'] >= prev_candle['low'] and prev2_candle['low'] >= prev_candle['low']) and latest_king_found and (prev_candle['low'] >= stop_loss_long):
                # Queen Candle Formed, Trail SL
                stop_loss_long = prev_candle['low']
                latest_king_found = False
        
            if (current_candle['low'] <= stop_loss_long):
                if current_candle['open'] < stop_loss_long:
                    if current_candle['datetime'] != entry_time_long:
                        exit_price_long = current_candle['open']
                        exit_time_long = df.iloc[i]['datetime']
                        points_long = exit_price_long - entry_price_long
                        # remarks_long = 'Gap SL'
                else:
                    exit_price_long = stop_loss_long
                    exit_time_long = df.iloc[i]['datetime']
                    points_long = exit_price_long - entry_price_long
                    # remarks_long = 'Initial SL Hit'
                
            # elif (current_candle['high'] >= target_long):
            #     exit_price_long = target_long
            #     exit_time_long = df.iloc[i]['datetime']
            #     points_long = exit_price_long - entry_price_long
            #     # remarks_long = 'Target Hit'
            
            if points_long:
                slippages = SLIPPAGE_ * (entry_price_long + exit_price_long)
                # qty = portfolio * index_lev / entry_price_long
                qty = portfolio * RPT_PCT / abs(entry_price_long - initial_sl_long)
                current_trade = {
                    'Type': 'LONG',
                    'Signal Generated At': signal_time_long,
                    'Entry Time': entry_time_long,
                    'Entry Price': entry_price_long,
                    'Initial SL': initial_sl_long,
                    'Final SL': stop_loss_long,
                    'Exit Time': exit_time_long,
                    'Exit Price': exit_price_long,
                    'Points': points_long,
                    'Slippages': slippages,
                    # 'Remarks': remarks_long,
                    'Qty': qty,
                    'PnL': points_long * qty,
                    'PnL w cs': (points_long - slippages) * qty,
                    # 'ROI%': points_long * qty * 100 / portfolio,
                    'ROI%': (points_long - slippages) * qty * 100 / portfolio,
                }
                trades.append(current_trade) 
                in_trade_long = False  
                king_candle = None  
                is_trailing_active_long = False
                points_long = None
                signal_time_long = None
                initial_sl_long = 100000

            
        # Check for Queen breakout (Short position)
        if queen_candle is not None and not in_trade_short and current_candle['low'] <= queen_candle['low']:
            
            if (current_candle['open'] < queen_candle['low'] and current_candle['high'] >= queen_candle['low']) or (current_candle['open'] >= queen_candle['low']):
                entry_price_short = queen_candle['low']
                stop_loss_short = king_candle['high'] if (king_candle is not None and king_candle['high'] > queen_candle['low']) else (max(last_candle['high'], prev_candle['high'], prev2_candle['high']))
                # stop_loss_short = entry_price_short * (1 + sl_pct)
                # stop_loss_short = initial_sl_short
                # target_short = entry_price_short * (1 - target_pct)
                entry_time_short = df.iloc[i]['datetime']
                in_trade_short = True
            elif current_candle['high'] < queen_candle['low']:
                queen_candle = None
                in_trade_short = False
            
        if in_trade_short:

            if last_candle['low'] >= prev_candle['low'] and prev2_candle['low'] >= prev_candle['low']:
                # Queen Candle Formed, Trail SL
                latest_queen_found = True
            
            if (last_candle['high'] <= prev_candle['high'] and prev2_candle['high'] <= prev_candle['high']) and latest_queen_found and (prev_candle['high'] <= stop_loss_short):
                # King Candle Formed, Trail SL
                stop_loss_short = prev_candle['high']
                latest_queen_found = False
            
            if (current_candle['high'] >= stop_loss_short):
                if current_candle['open'] > stop_loss_short:
                    if current_candle['datetime'] != entry_time_short:
                        exit_price_short = current_candle['open']
                        exit_time_short = df.iloc[i]['datetime']
                        points_short = entry_price_short - exit_price_short
                        # remarks_short = 'Gap SL'
                else:
                    exit_price_short = stop_loss_short
                    exit_time_short = df.iloc[i]['datetime']
                    points_short = entry_price_short - exit_price_short
                    # remarks_short = 'Initial SL Hit'
                
            # elif (current_candle['low'] <= target_short):
            #     exit_price_short = target_short
            #     exit_time_short = df.iloc[i]['datetime']
            #     points_short = entry_price_short - exit_price_short
            #     # remarks_short = 'Target Hit'
                
            if points_short:
                slippages = SLIPPAGE_ * (entry_price_short + exit_price_short)
                # qty = portfolio * index_lev / entry_price_short
                qty = portfolio * RPT_PCT / abs(entry_price_short - initial_sl_short)
                current_trade = {
                    'Type': 'SHORT',
                    'Signal Generated At': signal_time_short,
                    'Entry Time': entry_time_short,
                    'Entry Price': entry_price_short,
                    'Initial SL': initial_sl_short,
                    'Final SL': stop_loss_short,
                    'Exit Time': exit_time_short,
                    'Exit Price': exit_price_short,
                    'Points': points_short,
                    'Slippages': slippages,
                    # 'Remarks': remarks_short,
                    'Qty': qty,
                    'PnL': points_short * qty,
                    'PnL w cs': (points_short - slippages) * qty,
                    # 'ROI%': points_short * qty * 100 / portfolio,
                    'ROI%': (points_short - slippages) * qty * 100 / portfolio,
                }
                trades.append(current_trade)
                in_trade_short = False
                queen_candle = None
                is_trailing_active_short = False
                points_short = None
                signal_time_short = None
                initial_sl_short = 0
            
    return trades


# Example usage:
# trades = identify_trades(df)
# print(trades)


In [639]:
def positional2(df, sl_pct, target_pct):
    trades = []
    in_trade_long = False
    in_trade_short = False
    current_trade = None
    king_candle = None
    queen_candle = None
    slippage_pct = SLIPPAGE_
    portfolio = 10000000
    index_lev = 6
    entry_time = None
    exit_time = None
    is_trailing_active_long = False
    is_trailing_active_short = False
    points_long = None
    points_short = None
    initial_sl_long = 100000
    initial_sl_short = 0
    latest_king_found = False
    latest_queen_found = False
    long_trade_ready = False

    for i in range(3, len(df)):
        current_candle = df.iloc[i]
        n_1_candle = df.iloc[i-1]
        n_2_candle = df.iloc[i-2]
        n_3_candle = df.iloc[i-3]

        if (n_1_candle['high'] <= n_2_candle['high']) and (n_3_candle['high'] <= n_2_candle['high']):
            # King Candle Formed
            king_candle = n_2_candle

        if (n_1_candle['low'] >= n_2_candle['low']) and (n_3_candle['low'] >= n_2_candle['low']):
            # Queen Candle Formed
            queen_candle = n_2_candle

        if not in_trade_long:
            if king_candle is not None:
                if current_candle['high'] >= king_candle['high']:
                    # Go Long
                    if (current_candle['open'] > king_candle['high'] and current_candle['low'] <= king_candle['high']) or (current_candle['open'] <= king_candle['high']):
                        # Confirmation of Entry Triggered
                        entry_price_long = king_candle['high']
                        initial_sl_long = queen_candle['low']
                        stop_loss_long = initial_sl_long
                        entry_time_long = current_candle['datetime']
                        in_trade_long = True
                        signal_time_long = king_candle['datetime']
                        # long_trade_ready = True
                    else:
                        # Entry Skipped
                        in_trade_long = False

        if in_trade_long:
            
            if (n_1_candle['high'] <= n_2_candle['high']) and (n_3_candle['high'] <= n_2_candle['high']):
                latest_king_found = True

            if latest_king_found:
                if (n_1_candle['low'] >= n_2_candle['low']) and (n_3_candle['low'] >= n_2_candle['low']) and (n_2_candle['low'] > stop_loss_long):
                    # Queen Candle Formed, Trail SL
                    stop_loss_long = n_2_candle['low']

            if current_candle['low'] <= stop_loss_long:
                if (current_candle['open'] < stop_loss_long):
                    # SL Skipped
                    exit_price_long = current_candle['open']
                    in_trade_long = False
                    points_long = exit_price_long - entry_price_long
                    exit_time_long = current_candle['datetime']

                elif (current_candle['open'] >= stop_loss_long) and (current_candle['low'] <= stop_loss_long):
                    # SL Hit
                    exit_price_long = stop_loss_long
                    in_trade_long = False
                    points_long = exit_price_long - entry_price_long
                    exit_time_long = current_candle['datetime']

        if points_long:
            if entry_price_long > initial_sl_long:
                slippages = SLIPPAGE_ * (entry_price_long + exit_price_long)
                # qty = portfolio * index_lev / entry_price_long
                qty = portfolio * RPT_PCT / abs(entry_price_long - initial_sl_long)
                current_trade = {
                    'Type': 'LONG',
                    'Signal Generated At': signal_time_long,
                    'Entry Time': entry_time_long,
                    'Entry Price': entry_price_long,
                    'Initial SL': initial_sl_long,
                    'Final SL': stop_loss_long,
                    'Exit Time': exit_time_long,
                    'Exit Price': exit_price_long,
                    'Points': points_long,
                    'Slippages': slippages,
                    # 'Remarks': remarks_long,
                    'Qty': qty,
                    'PnL': points_long * qty,
                    'PnL w cs': (points_long - slippages) * qty,
                    # 'ROI%': points_long * qty * 100 / portfolio,
                    'ROI%': (points_long - slippages) * qty * 100 / portfolio,
                }
                trades.append(current_trade) 
            in_trade_long = False  
            king_candle = None  
            is_trailing_active_long = False
            points_long = None
            signal_time_long = None
            initial_sl_long = 100000
            # long_trade_ready = False

    return trades

In [640]:
# data_pandas.head(30)

In [641]:
tf = '60m'
ofs = '15m'
ma_period = 50

# data = resample(pl.DataFrame(bnf_1min), tf, ofs)
# data_pandas = data.to_pandas()
data_pandas = bnf_1min
data_pandas['ma'] = data_pandas['close'].rolling(ma_period).mean()
# data['ma'] = data['close'].rolling(ma_period).mean()
tb = positional2(data_pandas, 0.05, 1)

In [642]:
tb_pandas = pd.DataFrame(tb)
tb_pandas = tb_pandas.sort_values(by='Signal Generated At')

In [643]:
# tb_pandas = tb_pandas[tb_pandas['Type'] == 'LONG']

In [644]:
# tb_pandas['ROI% w cs'].sum()

In [645]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2015, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2015, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI% w cs"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI% w cs"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI% w cs"] > 0]["ROI% w cs"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI% w cs"] < 0]["ROI% w cs"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI% w cs"].cumsum() - year_trades["ROI% w cs"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI% w cs"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI% w cs"] > 0]["ROI% w cs"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI% w cs"] < 0]["ROI% w cs"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI% w cs"].cumsum() - combined_df_sorted["ROI% w cs"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [646]:
tb_pandas['Entry Time'] = pd.to_datetime(tb_pandas['Entry Time'])
tb_pandas['Trade Year'] = tb_pandas['Entry Time'].dt.year
tb_pandas['ROI% w cs'] = tb_pandas['ROI%']

In [647]:
stats = generate_stats(tb_pandas, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2015,-9.6568,47,25.5319,1.5830,-0.8187,-20.0833,-0.4808,...
2016,7.6917,52,36.5385,1.8769,-0.8475,-7.2074,1.0672,...
2017,-23.4923,66,25.7576,1.0407,-0.8405,-22.3543,-1.0509,...
2018,-6.0952,61,34.4262,1.1775,-0.7706,-18.1344,-0.3361,...
2019,-1.3586,60,31.6667,1.6047,-0.7768,-8.0423,-0.1689,...
2020,-10.9800,62,32.2581,1.0724,-0.7721,-10.6608,-1.0299,...
2021,5.8855,55,40.0000,1.5219,-0.8363,-9.9721,0.5902,...
2022,11.8698,51,37.2549,1.7293,-0.6558,-7.1912,1.6506,...
2023,-12.1830,57,28.0702,1.2029,-0.7666,-14.7158,-0.8279,...
2024,4.8286,45,37.7778,1.7186,-0.8710,-10.1253,0.4769,...


In [648]:
tb2 = tb_pandas[(tb_pandas['Trade Year'] <= 2016)]
tb2

,Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points,Slippages,Qty,PnL,PnL w cs,ROI%,Trade Year,ROI% w cs
0,LONG,2015-03-03 17:00:00+05:30,2015-03-04 09:00:00+05:30,170.2000,165.4000,171.9000,2015-03-09 13:00:00+05:30,171.9000,1.7000,0.3421,20833.3333,35416.6667,28289.5833,0.2829,2015,0.2829
1,LONG,2015-03-10 21:00:00+05:30,2015-03-11 17:00:00+05:30,173.5000,169.6000,169.6000,2015-03-11 17:00:00+05:30,169.6000,-3.9000,0.3431,25641.0256,-100000.0000,-108797.4359,-1.0880,2015,-1.0880
2,LONG,2015-03-11 21:00:00+05:30,2015-03-12 17:00:00+05:30,179.4000,168.5000,174.8000,2015-03-12 21:00:00+05:30,174.8000,-4.6000,0.3542,9174.3119,-42201.8349,-45451.3761,-0.4545,2015,-0.4545
3,LONG,2015-03-13 21:00:00+05:30,2015-03-17 13:00:00+05:30,174.5000,169.4000,175.0000,2015-03-19 17:00:00+05:30,175.0000,0.5000,0.3495,19607.8431,9803.9216,2950.9804,0.0295,2015,0.0295
4,LONG,2015-03-30 09:00:00+05:30,2015-03-30 17:00:00+05:30,167.5000,166.1000,166.1000,2015-03-30 17:00:00+05:30,165.6000,-1.9000,0.3331,71428.5714,-135714.2857,-159507.1429,-1.5951,2015,-1.5951
5,LONG,2015-03-30 17:00:00+05:30,2015-03-31 17:00:00+05:30,168.6000,164.7000,165.3000,2015-04-01 13:00:00+05:30,165.3000,-3.3000,0.3339,25641.0256,-84615.3846,-93176.9231,-0.9318,2015,-0.9318
6,LONG,2015-04-01 13:00:00+05:30,2015-04-02 17:00:00+05:30,167.8000,162.2000,166.1000,2015-04-08 13:00:00+05:30,166.1000,-1.7000,0.3339,17857.1429,-30357.1429,-36319.6429,-0.3632,2015,-0.3632
7,LONG,2015-04-13 17:00:00+05:30,2015-04-14 17:00:00+05:30,159.4000,155.7000,161.8000,2015-04-20 17:00:00+05:30,161.8000,2.4000,0.3212,27027.0270,64864.8649,56183.7838,0.5618,2015,0.5618
8,LONG,2015-04-21 17:00:00+05:30,2015-04-22 13:00:00+05:30,164.0000,160.7000,163.1000,2015-04-23 17:00:00+05:30,163.1000,-0.9000,0.3271,30303.0303,-27272.7273,-37184.8485,-0.3718,2015,-0.3718
9,LONG,2015-04-28 17:00:00+05:30,2015-04-29 17:00:00+05:30,162.2000,160.1000,176.5000,2015-05-06 13:00:00+05:30,176.5000,14.3000,0.3387,47619.0476,680952.3810,664823.8095,6.6482,2015,6.6482


In [593]:
tb2['Points'].sum()

-22.19999999999999

# Change No. 1 :
## Alternate King and Queen candle for trailing

# Change No. 2 :
## Confirmation to be done based on candle closing. CHECK FORWARD BIAS